In [1]:
from scipy.io import loadmat
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import time

from sdf.robot_sdf import RobotSdfCollisionNet

In [2]:
device = torch.device('cpu')
tensor_args = {'device': device, 'dtype': torch.float32}
data = loadmat('../data-sampling/datasets/data_mesh.mat')['dataset']
print(data.shape)

(4950000, 19)


In [3]:
L1 = 0
L2 = int(1.0 * data.shape[0])
n_size = L2
train_ratio = 0.98
test_ratio = 0.01
val_ratio = 1 - train_ratio - test_ratio
idx_train = np.arange(0, int(n_size * train_ratio))
idx_val = np.arange(idx_train[-1] + 1, int(n_size * (train_ratio + test_ratio)))
idx_test = np.arange(idx_val[-1] + 1, int(n_size))

print("Number of train data      : ", idx_train.shape[0])
print("Number of validation data : ", idx_val.shape[0])
print("Number of test data       : ", idx_test.shape[0])

Number of train data      :  4851000
Number of validation data :  49500
Number of test data       :  49500


In [4]:
x = torch.Tensor(data[L1:L2, 0:10]).to(device, dtype=torch.float16)
y = 100 * torch.Tensor(data[L1:L2, 10:]).to(device, dtype=torch.float16)
dof = x.shape[1]
s = 256
n_layers = 5
batch_size = 50000
skips = []
# fname = 'sdf_%dx%d_mesh_%d_drop_%1f.pt'%(s,n_layers,batch_size, p)
fname = 'model/sdf_%dx%d_mesh_%d.pt'%(s,n_layers,batch_size)
if skips == []:
    n_layers-=1
nn_model = RobotSdfCollisionNet(in_channels=dof, out_channels=y.shape[1], layers=[s] * n_layers, skips=skips, dropout_ratio=0)
nn_model.load_weights(fname, tensor_args)
nn_model.model.to(**tensor_args)
model = nn_model.model

Weights loaded!


In [5]:
x_test = x[idx_test, :]
y_test = y[idx_test, :]
x_test

tensor([[-2.3340,  1.2070, -2.9668,  ...,  0.0271, -0.0393,  0.1138],
        [-2.3340,  1.2070, -2.9668,  ..., -0.1213, -0.0626,  0.0052],
        [-2.3340,  1.2070, -2.9668,  ..., -0.0793, -0.0112,  0.0179],
        ...,
        [-0.4402,  1.6855,  2.9668,  ..., -0.1785,  0.0033,  0.4771],
        [-0.4402,  1.6855,  2.9668,  ..., -0.1196, -0.1043,  0.5439],
        [-0.4402,  1.6855,  2.9668,  ..., -0.1323, -0.0710,  0.5693]],
       dtype=torch.float16)

In [6]:
# x_tmp = torch.from_numpy( np.array([x_test[0,:].cpu().detach().numpy()], dtype = np.float32) )
# print(x_tmp)
x_tmp = np.zeros((49500,1,10), dtype = np.float32)
for i in range(0,49500):
    x_tmp[i] = np.array([x_test[i,:].cpu().detach().numpy()], dtype = np.float32)
x_tmp = torch.from_numpy(x_tmp)
print(x_tmp)

tensor([[[-2.3340,  1.2070, -2.9668,  ...,  0.0271, -0.0393,  0.1138]],

        [[-2.3340,  1.2070, -2.9668,  ..., -0.1213, -0.0626,  0.0052]],

        [[-2.3340,  1.2070, -2.9668,  ..., -0.0793, -0.0112,  0.0179]],

        ...,

        [[-0.4402,  1.6855,  2.9668,  ..., -0.1785,  0.0033,  0.4771]],

        [[-0.4402,  1.6855,  2.9668,  ..., -0.1196, -0.1043,  0.5439]],

        [[-0.4402,  1.6855,  2.9668,  ..., -0.1323, -0.0710,  0.5693]]])


In [7]:
tic=time.time()
for i in range(21000):
    # y_pred, j_pred, _ = nn_model.compute_signed_distance_wgrad(x_tmp[i])
    y_pred = nn_model.compute_signed_distance(x_tmp[i])
toc=time.time()
# print(y_pred)
# print(j_pred)
print(toc-tic)

1.982814073562622


In [8]:
import libNJSDF_FUN as NJSDF_FUN
NJSDF_FUN.setNeuralNetwork()
x_tmp2 = x_test.cpu().detach().numpy()
print(x_tmp2)

Thread: 24
[[-2.334     1.207    -2.967    ...  0.02711  -0.0393    0.11383 ]
 [-2.334     1.207    -2.967    ... -0.12134  -0.0626    0.005226]
 [-2.334     1.207    -2.967    ... -0.0793   -0.01122   0.01785 ]
 ...
 [-0.4402    1.686     2.967    ... -0.1785    0.003325  0.477   ]
 [-0.4402    1.686     2.967    ... -0.11957  -0.1043    0.544   ]
 [-0.4402    1.686     2.967    ... -0.1323   -0.071     0.5693  ]]


In [9]:
tic_=time.time()
for i in range(21000):
    NJSDF_FUN.setNetworkInput(x_tmp2[i])
    # g, g_d = NJSDF_FUN.calculateMlpOutput_wgrad(False)
    g= NJSDF_FUN.calculateMlpOutput()
toc_=time.time()
# print(g)
# print(g_d)
print(toc_-tic_)

0.4367852210998535


In [10]:
from multiprocessing import Process

def SDF_python(input):
    y_pred = nn_model.compute_signed_distance(input)

def SDF_cpp(input):
    NJSDF_FUN.setNetworkInput(input)
    g = NJSDF_FUN.calculateMlpOutput()

In [11]:
processes_python = [Process(target=SDF_python, args=(x_tmp[i])) for i in range(21000)]
processes_cpp    = [Process(target=SDF_cpp, args=(x_tmp2[i])) for i in range(21000)]

In [12]:
# tic_=time.time()
# for process in processes_python:
#         process.start()
# # wait for all processes to complete
# for process in processes_python:
#     process.join()
# toc_=time.time()
# print(toc_-tic_)

In [13]:
# tic_=time.time()
# for process in processes_cpp:
#         process.start()
# # wait for all processes to complete
# for process in processes_cpp:
#     process.join()
# toc_=time.time()
# print(toc_-tic_)